## Installation

Note : we are pinning the inspect_ai due to a recent breaking change

In [ ]:
%pip install -q openai anthropic ipywidgets colorama
import os
os.environ['XDG_RUNTIME_DIR']="/tmp"
from helpers.reporter.pretty import pretty_results


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Run a shell one with tools

We add all the models we want to test across

In [9]:
from inspect_ai.agent import Agent, AgentState, agent, as_solver, as_tool
from inspect_ai.model import ChatMessageSystem, get_model
from inspect_ai.tool import web_browser

@agent
def web_surfer() -> Agent:
    async def execute(state: AgentState) -> AgentState:
        """Web research assistant."""
      
        # some general guidance for the agent
        state.messages.append(
            ChatMessageSystem(
                content="You are an expert at using a " + 
                "web browser to answer questions."
            )
        )

        # run a tool loop w/ the web_browser 
        messages, output = await get_model().generate_loop(
            state.messages, tools=web_browser(interactive=False),
        )

        # update and return state
        state.output = output
        state.messages.extend(messages)
        return state

    return execute

from inspect_ai.agent import react
from inspect_ai.tool import web_browser


from inspect_ai.solver import generate, use_tools
from inspect_ai import Task, eval, task
from inspect_ai.dataset import json_dataset
from inspect_ai.scorer import includes
from inspect_ai.dataset import Sample

@task
def myagent():

    dataset = [
     Sample(input="Where does patrick debois live ? Use wikipedia", target="Belgium")
    ]
 
    task = Task(
        dataset=dataset,
        solver=web_surfer(),
        scorer=includes(),
        sandbox="docker"
    )
    return task

results7 = eval(myagent, max_steps=10,log_level="info",display="none")
p7 = pretty_results(results7)
print(p7)

Output()


Status: success Model: openai/gpt-4o-mini
/workspaces/workshop-testing/lessons/04-testing/logs/2025-05-06T19-53-13+00-00_myagent_AaZvaU8VySqsRwtmasTGzJ.eval
======= Sample ========
input : Where does patrick debois live ? Use wikipedia
target: Belgium
 user       > Where does patrick debois live ? Use wikipedia
 system     > You are an expert at using a web browser to answer questions.
 assistant [tool:web_browser_go] > {'url': 'https://en.wikipedia.org/wiki/Patrick_Debois'}
 assistant  > 
 tool[web_browser_go] +> main content:
Patrick Debois Patrick Debois This article is only available in this language. Add the article for other languages Namespaces Page tools Tools Appearance Appearance hide hide Patrick Debois From Wikipedia, the free encyclopedia  Look for Patrick Debois on one of Wikipedia's sister projects sister projects : Wiktionary Wiktionary (dictionary) Wikibooks Wikibooks (textbooks) Wikiquote Wikiquote (quotations) Wikisource Wikisource (library) Wikiversity Wikiversity 

## Handoff

In [ ]:
from inspect_ai.agent import react
from inspect_ai.tool import web_browser

web_surfer = react(
    name="web_surfer",
    description="Web research assistant",
    prompt="You are a tenacious web researcher that is expert "
           + "at using a web browser to answer questions.",
    tools=web_browser()   
)

from inspect_ai.agent import handoff
from inspect_ai.dataset import Sample

supervisor = react(
    prompt="You are an agent that can answer addition " 
            + "problems and do web research.",
    tools=[ handoff(web_surfer)]
)

task2 = Task(
    dataset=[
        Sample(input="Please add 1+1 then tell me what " 
                     + "movies were popular in 2020")
    ],
    solver=supervisor,
    sandbox="docker",
)

results8 = eval(task2, max_steps=10,log_level="info",display="none")
p8 = pretty_results(results8)
print(p8)

Output()